# Utiliser des environnements


Quand vous exécutez un script en tant que travail Azure Machine Learning, vous devez définir son contexte d’exécution. Une configuration clé est la cible de calcul sur laquelle le script sera exécuté. Il peut s’agir de la station de travail locale (dans ce cas, l’instance de calcul) ou d’une cible de calcul distante telle que le cluster de calcul managé Azure Machine Learning qui est provisionné à la demande.

Dans ce notebook, vous créez un cluster de calcul et vous explorez les cibles de calcul pour les travaux.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Exécuter un script en tant que travail

Pour entraîner un modèle, vous allez d’abord créer le script **diabetes_training.py** dans le dossier **src**. Le script utilise le fichier **diabetes.csv** dans le même dossier que les données d’entraînement.

Notez que vous importez des bibliothèques au début du script. Les fonctions fournies dans ces bibliothèques sont utilisées pour le traitement des données et l’entraînement du modèle. Ces bibliothèques doivent toujours être installées, quelle que soit la capacité de calcul que vous utilisez pour exécuter le script.

In [ ]:
Après avoir créé le script, vous pouvez l’exécuter en tant que travail. Le script utilise des bibliothèques courantes. Vous pouvez donc utiliser un environnement organisé comme pandas, numpy et scikit-learn, entre autres.

Le travail utilise la dernière version de l’environnement organisé : `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`.

In [ ]:
Pendant l’exécution du travail, vous pouvez déjà exécuter les cellules suivantes.

## Liste des environnements

Explorons les environnements au sein de l’espace de travail. 

Dans le travail précédent, vous avez utilisé l’un des environnements organisés. Pour explorer tous les environnements qui existent déjà dans l’espace de travail, vous pouvez les lister : 

In [ ]:
Notez que tous les environnements organisés ont des noms qui commencent par **AzureML-** (vous ne pouvez pas utiliser ce préfixe pour vos propres environnements).

Pour examiner un environnement spécifique, vous pouvez récupérer l’environnement par son nom et sa version. Par exemple, vous pouvez récupérer la *description* et les *étiquettes* de l’environnement organisé que vous avez utilisé pour le travail précédent :

## Créer et utiliser un environnement personnalisé

Si un environnement organisé n’inclut pas tous les packages Python dont vous avez besoin pour exécuter votre script, vous pouvez créer votre propre environnement personnalisé. En listant tous les packages nécessaires dans un environnement, vous pouvez facilement réexécuter vos scripts. Toutes les dépendances sont stockées dans l’environnement que vous pouvez ensuite spécifier dans la configuration du travail, indépendamment de la capacité de calcul que vous utilisez.

Par exemple, vous pouvez créer un environnement simplement à partir d’une image Docker. Certains frameworks tels que PyTorch auront une image Docker publique incluant déjà tout ce dont vous avez besoin. 

Nous allons créer un environnement à partir d’une image Docker :

In [ ]:
L’environnement est désormais inscrit dans votre espace de travail. Vous pouvez donc le référencer lorsque vous exécutez un script en tant que travail :

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Le travail échouera vite ! Lisez le message d’erreur. </p>

Le message d’erreur vous indique qu’il n’y a aucun module nommé pandas. Cette erreur a deux causes possibles :

- Le script utilise pandas, mais n’a pas importé la bibliothèque (`import pandas as pd`). 
- Le script importe la bibliothèque au début du script, mais la bibliothèque n’est pas installée sur la capacité de calcul (`pip install pandas`).

Vous vérifiez le script `diabetes-training.py` et constatez qu’il est correct, ce qui signifie que la bibliothèque n’a pas été installée. En d’autres termes, l’environnement n’a pas inclus les packages nécessaires.

Créons un environnement à l’aide de l’image Docker de base qui a été utilisée dans le travail précédent. Maintenant, vous allez ajouter une spécification conda pour vous assurer que les packages nécessaires seront installés. Tout d’abord, exécutez la cellule suivante pour créer le fichier de spécification conda :

Notez que toutes les dépendances nécessaires sont incluses dans le fichier de spécification conda afin de permettre la bonne exécution du script.

Créez un environnement à l’aide de l’image Docker de base **et** du fichier de spécification conda pour ajouter les dépendances nécessaires. Azure Machine Learning crée l’environnement conda sur l’image Docker que vous avez fournie. 

In [ ]:
Vous pouvez maintenant soumettre un travail avec le nouvel environnement pour exécuter le script :

La soumission du travail avec le nouvel environnement personnalisé déclenche la création de l’environnement. La première fois que vous utilisez un nouvel environnement, la création de l’environnement peut prendre entre 10 et 15 minutes, ce qui augmente aussi la durée du travail. 

Vous pouvez également choisir de déclencher manuellement la création de l’environnement avant de soumettre un travail. L’environnement doit être créé uniquement la première fois que vous l’utilisez. 

Lorsque le travail déclenche la création d’un environnement, vous pouvez consulter les journaux de la build sous l’onglet **Sorties + journaux** du travail. Ouvrez **azureml-logs/20_image_build_log.txt** pour inspecter les journaux de la build d’environnement. 

![Capture d’écran des journaux de la build](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)